In [ ]:
#insecure eval

user_input=input("Enter expression: ")
result = eval(user_input)
print("Result: ", result)


# safe_eval_arith.py
import ast
import operator

# mapping AST operators to functions
operators = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow,
    ast.USub: operator.neg,
}

def eval_expr(node):
    if isinstance(node, ast.Num):  # < Py3.8
        return node.n
    if isinstance(node, ast.Constant):  # Python 3.8+
        return node.value
    if isinstance(node, ast.BinOp):
        left = eval_expr(node.left)
        right = eval_expr(node.right)
        op_type = type(node.op)
        if op_type in operators:
            return operators[op_type](left, right)
    if isinstance(node, ast.UnaryOp):
        operand = eval_expr(node.operand)
        op_type = type(node.op)
        if op_type in operators:
            return operators[op_type](operand)
    raise ValueError("Unsupported expression")

expr = input("Enter arithmetic expression: ")
parsed = ast.parse(expr, mode="eval")
result = eval_expr(parsed.body)
print("Result:", result)



Result:  5


In [10]:
# insecure_sql.py
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, pass TEXT)")
c.execute("INSERT INTO users (name, pass) VALUES ('alice', 'password123')")
c.execute("INSERT INTO users (name, pass) VALUES ('amrit', 'password456')")


username = input("Username: ")
query = "SELECT * FROM users WHERE name = '%s'" % username
c.execute(query)
print(c.fetchall())



# safe_sql.py
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, pass TEXT)")
c.execute("INSERT INTO users (name, pass) VALUES (?, ?)", ("alice", "password123"))

username = input("Username: ")
# the ? placeholder ensures parameters are bound safely
c.execute("SELECT * FROM users WHERE name = ?", (username,))
print(c.fetchall())



[(1, 'alice', 'password123'), (2, 'amrit', 'password456')]
[]


In [3]:
# insecure_cmd.py
import subprocess

filename = input("Enter filename to list: ")
# Dangerous: the Windows shell interprets special characters
subprocess.run(f"dir {filename}", shell=True)


# safe_cmd.py
import subprocess

filename = input("Enter filename to list: ")
# safe: pass args as a list and avoid shell=True
subprocess.run(["cmd", "/c", "dir", filename])


CompletedProcess(args=['cmd', '/c', 'dir', 'securee.ipynb & del D:\\Work\\Security\\5006\\mcp\\secure\\sample1.txt'], returncode=1)

In [ ]:
# insecure_pickle.py
import pickle

data = input("Paste pickled data (base64): ")
import base64
raw = base64.b64decode(data)
obj = pickle.loads(raw)  # Dangerous: executing untrusted pickle
print("Loaded object:", obj)



# safe_deser.py
import json

data = input("Paste JSON data: ")
obj = json.loads(data)
print("Loaded object:", obj)

In [ ]:
# insecure_upload.py (Flask)
from flask import Flask, request

app = Flask(__name__)

@app.route("/upload", methods=["POST"])
def upload():
    f = request.files['file']
    # Dangerous: directly using uploaded filename
    f.save("D:\\Work\\Security\\5006\\mcp\\secure" + f.filename)
    return "Saved"


# safe_upload.py (Flask)
from flask import Flask, request
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
UPLOAD_DIR = "/var/www/uploads"

@app.route("/upload", methods=["POST"])
def upload():
    f = request.files['file']
    filename = secure_filename(f.filename)  # sanitizes filename
    if not filename:
        return "Invalid filename", 400
    # ensure upload dir exists and join path safely
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    dest = os.path.join(UPLOAD_DIR, filename)
    f.save(dest)
    return "Saved"



In [ ]:
# insecure_password.py
password = input("Create password: ")
# Danger: saving plain text
with open("sample1.txt", "a") as f:
    f.write("alice:" + password + "\n")



# safe_password.py
import bcrypt

password = input("Create password: ").encode()
# generate salt and store hashed password
hashed = bcrypt.hashpw(password, bcrypt.gensalt())

# store hashed (binary) as base64 or decode to save as text
import base64
stored = base64.b64encode(hashed).decode()

# save stored to file or DB
with open("users.txt", "a") as f:
    f.write("alice:" + stored + "\n")

# to verify later
attempt = input("Enter password to check: ").encode()
# read stored and decode from base64 then verify
with open("users.txt") as f:
    lines = f.readlines()
stored = lines[-1].split(":",1)[1].strip()
hashed = base64.b64decode(stored)
if bcrypt.checkpw(attempt, hashed):
    print("Password OK")
else:
    print("Wrong password")



In [ ]:
# insecure_xss.py (Flask)
from flask import Flask, request
app = Flask(__name__)

@app.route("/greet", methods=["GET"])
def greet():
    name = request.args.get("name", "guest")
    return f"<h1>Hello {name}</h1>"


# safe_xss.py (Flask template)
from flask import Flask, request, render_template_string
app = Flask(__name__)

template = "<h1>Hello {{ name }}</h1>"

@app.route("/greet", methods=["GET"])
def greet():
    name = request.args.get("name", "guest")
    # render_template_string will auto-escape name
    return render_template_string(template, name=name)


Do not eval() untrusted input — parse safely.
Use parameterized queries for SQL.
Avoid shell=True and pass command args as lists.
Never unpickle untrusted data; use JSON or signed formats.
Sanitize filenames and prevent path traversal.
Hash passwords with bcrypt/argon2; never store plain text.
Keep secrets out of code; use env vars or secret managers.
Escape HTML output to prevent XSS.
Don’t log secrets.
Run services with least privilege.